#Tópicos Avanzados de Computación II
##Dr. Carlos Villaseñor
### Lección 5 -  Entrenando Redes Neuronales Profundas parte I






In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import datetime

In [4]:
raw_dataset = pd.read_csv('./DataSets/cancer.csv')

dataset = raw_dataset.copy()
print(dataset)

     Clump  UnifSize  UnifShape  MargAdh  SingEpiSize  BareNuc  BlandChrom  \
0        5         1          1        1            2        1           3   
1        5         4          4        5            7       10           3   
2        3         1          1        1            2        2           3   
3        6         8          8        1            3        4           3   
4        4         1          1        3            2        1           3   
..     ...       ...        ...      ...          ...      ...         ...   
678      3         1          1        1            3        2           1   
679      2         1          1        1            2        1           1   
680      5        10         10        3            7        3           8   
681      4         8          6        4            3        4          10   
682      4         8          8        5            4        5          10   

     NormNucl  Mit  Class  
0           1    1      1  
1      

In [5]:
dataset.isna().sum()

Clump          0
UnifSize       0
UnifShape      0
MargAdh        0
SingEpiSize    0
BareNuc        0
BlandChrom     0
NormNucl       0
Mit            0
Class          0
dtype: int64

In [6]:
dataset = dataset.dropna()

In [7]:
dataset.tail()

,Clump,UnifSize,UnifShape,MargAdh,SingEpiSize,BareNuc,BlandChrom,NormNucl,Mit,Class
678,3,1,1,1,3,2,1,1,1,1
679,2,1,1,1,2,1,1,1,1,1
680,5,10,10,3,7,3,8,10,2,0
681,4,8,6,4,3,4,10,6,1,0
682,4,8,8,5,4,5,10,4,1,0


In [8]:
x = np.asanyarray(dataset.drop(columns=['Class']))
y = np.asanyarray(dataset[['Class']])
x = StandardScaler().fit_transform(x)
print(x.shape)
print(y.shape)

(683, 9)
(683, 1)


In [9]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=0)

Ahora vamos a crear una función que nos regrese una red neuronal, al igual que la práctica anterior vamos a crear una red densa, pero vamos a agregar unos parámetros que serán intercambiados por el algoritmo de búsqueda de hiperparámetros.

In [10]:
def model_builder(hp):

  model = keras.Sequential()

  # Create Hyperparameters Space
  hp_activation = hp.Choice('dense_activation',
                       values=['relu', 'tanh', 'sigmoid'])
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)

  # Neural Layer
  model.add(keras.layers.Dense(units = hp_units,
                               activation = hp_activation,
                               input_shape=[x.shape[1]]))

  # Output layer
  model.add(keras.layers.Dense(1, activation='sigmoid'))

  # Create Hyperparameter Space
  hp_learning_rate = hp.Float('learning_rate', min_value=1e-5, max_value=1e-2,
                      sampling='LOG',  default=1e-3)

  # Compile model
  model.compile(loss='binary_crossentropy',
                optimizer=keras.optimizers.Adam(learning_rate = hp_learning_rate),
                metrics=['accuracy'])
  return  model

Ahora vamos a crear nuestro objeto tuner (ajustador), el cual se encargará de crear un modelo de red neuronal con ciertos hiperparámetros y la entrenará. Corra solo uno de los siguientes tres bloques de código

In [16]:
tuner = kt.RandomSearch(model_builder,
                        objective='val_accuracy',
                        max_trials=20,
                        directory='Results',
                        project_name='Random')

In [ ]:
tuner = kt.BayesianOptimization(model_builder,
                                objective='val_accuracy',
                                max_trials=20,
                                directory='Results',
                                project_name='Bayesian')

In [11]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=20,
                     directory='Results',
                     project_name='HyperBand')

INFO:tensorflow:Reloading Oracle from existing project Results\HyperBand\oracle.json
INFO:tensorflow:Reloading Tuner from Results\HyperBand\tuner0.json


Vamos a crear el siguiente Callback, para borra al final de entrenar un modelo

In [12]:
import IPython
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

Vamos a correr las pruebas

In [13]:
tuner.search(xtrain, ytrain, epochs=20, validation_split=0.15,
             callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')} and dense activation is {best_hps.get('dense_activation')}
""")

INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 416 and the optimal learning rate for the optimizer
is 0.008452433545678683 and dense activation is relu



Finalmente vamos a recrear el modelo que consiguió el mejor desempeño

In [14]:
log_dir = "DL_L06_A06/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
history = model.fit(xtrain, ytrain, epochs = 50, validation_data = (xtest, ytest), verbose=1, callbacks=[tensorboard_callback])

Epoch 1/50
18/18 [==============================] - 1s 22ms/step - loss: 0.1545 - accuracy: 0.9414 - val_loss: 0.0935 - val_accuracy: 0.9562
Epoch 2/50
18/18 [==============================] - 0s 8ms/step - loss: 0.0873 - accuracy: 0.9762 - val_loss: 0.0727 - val_accuracy: 0.9562
Epoch 3/50
18/18 [==============================] - 0s 7ms/step - loss: 0.0699 - accuracy: 0.9762 - val_loss: 0.0784 - val_accuracy: 0.9708
Epoch 4/50
18/18 [==============================] - 0s 7ms/step - loss: 0.0656 - accuracy: 0.9762 - val_loss: 0.0700 - val_accuracy: 0.9635
Epoch 5/50
18/18 [==============================] - 0s 7ms/step - loss: 0.0600 - accuracy: 0.9799 - val_loss: 0.0649 - val_accuracy: 0.9781
Epoch 6/50
18/18 [==============================] - 0s 7ms/step - loss: 0.0633 - accuracy: 0.9744 - val_loss: 0.0506 - val_accuracy: 0.9781
Epoch 7/50
18/18 [==============================] - 0s 7ms/step - loss: 0.0622 - accuracy: 0.9762 - val_loss: 0.0520 - val_accuracy: 0.9781
Epoch 8/50
18/18 [=

In [15]:
score = model.evaluate(xtest, ytest)

5/5 [==============================] - 0s 2ms/step - loss: 0.1222 - accuracy: 0.9708


In [16]:
log_dir = "DL_L06_A06/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback_es = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1) 

es = keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=20)

# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
history = model.fit(xtrain, ytrain, epochs = 50, validation_data = (xtest, ytest), verbose=1, callbacks=[es, tensorboard_callback_es])

score = model.evaluate(xtest, ytest)

Epoch 1/50
18/18 [==============================] - 1s 17ms/step - loss: 0.1570 - accuracy: 0.9231 - val_loss: 0.0950 - val_accuracy: 0.9635
Epoch 2/50
18/18 [==============================] - 0s 7ms/step - loss: 0.0945 - accuracy: 0.9744 - val_loss: 0.0704 - val_accuracy: 0.9635
Epoch 3/50
18/18 [==============================] - 0s 7ms/step - loss: 0.0688 - accuracy: 0.9744 - val_loss: 0.0658 - val_accuracy: 0.9781
Epoch 4/50
18/18 [==============================] - 0s 9ms/step - loss: 0.0664 - accuracy: 0.9780 - val_loss: 0.0616 - val_accuracy: 0.9708
Epoch 5/50
18/18 [==============================] - 0s 10ms/step - loss: 0.0605 - accuracy: 0.9780 - val_loss: 0.0622 - val_accuracy: 0.9708
Epoch 6/50
18/18 [==============================] - 0s 7ms/step - loss: 0.0575 - accuracy: 0.9799 - val_loss: 0.0595 - val_accuracy: 0.9708
Epoch 7/50
18/18 [==============================] - 0s 7ms/step - loss: 0.0670 - accuracy: 0.9780 - val_loss: 0.0470 - val_accuracy: 0.9854
Epoch 8/50
18/18 [

In [17]:
%load_ext tensorboard
%tensorboard --logdir DL_L06_A06/logs/fit